# Bond Options Black Karasinski Model

In this notebook I create a bond option and value it using the Black-Karasinski model

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [3]:
from financepy.finutils.FinDate import FinDate
from financepy.models.FinModelRatesBK import FinModelRatesBK
from financepy.finutils.FinHelperFunctions import printTree
from financepy.market.curves.FinDiscountCurve import FinDiscountCurve
from financepy.products.bonds.FinBond import FinBond
from financepy.finutils.FinFrequency import FinFrequencyTypes
from financepy.finutils.FinDayCount import FinDayCountTypes
from financepy.finutils.FinGlobalVariables import gDaysInYear

# American Bond Option Valuation

We value an American style option on a coupon paying bond.

In [4]:
settlementDate = FinDate(1, 12, 2019)

The bond pays 5% semi-annually for 10 years.

In [5]:
maturityDate = settlementDate.addTenor("10Y")
coupon = 0.05
frequencyType = FinFrequencyTypes.SEMI_ANNUAL
accrualType = FinDayCountTypes.ACT_ACT_ICMA

In [6]:
bond = FinBond(maturityDate, coupon, frequencyType, accrualType)

In [7]:
bond.calculateFlowDates(settlementDate)

In [8]:
couponTimes = []
couponFlows = []
cpn = bond._coupon/bond._frequency
for flowDate in bond._flowDates:
    flowTime = (flowDate - settlementDate) / gDaysInYear
    couponTimes.append(flowTime)
    couponFlows.append(cpn)

couponTimes = np.array(couponTimes)
couponFlows = np.array(couponFlows)

The option expires in 18 months with a strike of 105

In [9]:
expiryDate = settlementDate.addTenor("18m")
texp = (expiryDate - settlementDate) / gDaysInYear
strikePrice = 105.0
face = 100.0

We set the BK model to have 20% of lognormal rate volatility and 0.05 of mean-reversion

In [10]:
sigma = 0.20
a = 0.05

We set up a flat discount curve at 5%

In [11]:
tmat = (maturityDate - settlementDate) / gDaysInYear
times = np.linspace(0, tmat, 20)
dfs = np.exp(-0.05*times)

In [12]:
discountCurve = FinDiscountCurve(settlementDate, times, dfs)

We can price the basic bond without any option

In [15]:
price = bond.valueBondUsingDiscountCurve(settlementDate, discountCurve)

In [16]:
print("Bond Price:", price)

Bond Price: 99.50391846904128


Now we price the option - the tree only needs to go out to the expiry date of the option

Test convergence

In [17]:
numStepsList = [100, 150, 200, 250, 300, 400, 500, 600, 700, 800, 900, 1000]

In [20]:
treeVector = []
for numTimeSteps in numStepsList:
    model = FinModelRatesBK(a, sigma, numTimeSteps)
    model.buildTree(tmat, times, dfs)
    vOption, vBond = model.bondOption(texp, strikePrice, face, couponTimes, couponFlows, True)
    treeVector.append(vOption)
    print(numTimeSteps, vOption, vBond)

plt.plot(numStepsList, treeVector);    

FinError: Coupon times are the same

The option price converges towards something close to the price of 0.699 mentioned in the book by Hull (Fig 28.11). There is a convergence issue at 300 steps with the bond price that needs to be examined. It relates to a numerical issue regarding the coupon payment and expiry date being close together.